# Cummings OCR

In [1]:
# Libraries
from pathlib import Path
import os
import sys
import csv
import shutil
import itertools
import random
import PIL
import pytesseract
import numpy as np
import scipy.ndimage as nd
import pandas as pd
from collections import Counter
from PIL import Image, ImageChops, ImageStat
from random import sample
import re
import glob
from datetime import datetime
pytesseract.pytesseract.tesseract_cmd = r"/usr/bin/tesseract"

In [2]:
sys.path.append("/home/lucas-jerusalimiec/Documents/OCR Bin/Split")
from cropfunctions  import rotation_angle, trim, get_bands, simp_bd, combine_bbox
from ocr_func import cutMarg, OCRtestImg, testList, tsvOCR, adjustImg

In [3]:
# Display a list of languages in their 3-letter codes supported by Tesseract.
print(pytesseract.get_languages(config=''))

['eng', 'fra', 'frm', 'lat', 'osd']


In [4]:
# Create Directories
doc_name = 'Démonomanie I.1'
output_folder = Path('./output')
output_folder.mkdir(exist_ok=True)
outDir = "./output"
jpgs_folder = Path('./images')
dirpath = "./images"

In [8]:
# List of metadata dictionaries
file_count = len([name for name in os.listdir(jpgs_folder) if os.path.isfile(os.path.join(jpgs_folder, name))])
limit = len([name for name in os.listdir(jpgs_folder) if os.path.isfile(os.path.join(jpgs_folder, name))])
side = ["right" if i % 2 == 0 else "left" for i in range(file_count)]
meta = []
count = 0
# For each jpeg file in the rotated folder, do the following:
for jpg in itertools.islice(sorted(jpgs_folder.rglob('*.jpg')), limit):
        row_dict = dict()
        row_dict["filename"] = jpg.name
        row_dict["side"] = side[count]
        row_dict["start_section"] = False
        row_dict["find_top"] = False
        row_dict["background"] = 0
        row_dict["bbox"] = []
        meta.append(row_dict)
        count = count+1
            
print(meta[0])

{'filename': 'Démonomanie I.1_image_0000.jpg', 'side': 'right', 'start_section': False, 'find_top': False, 'background': 0, 'bbox': []}


In [9]:
# Extract find_top
find_top = [d['find_top'] for d in meta]
print(find_top[0:3])

[False, False, False]


In [10]:
#Combined rotation, greyscale (inverse), and trim workflow metadata
### Convert multiple jpegs into a set of greyscale images ###
limit = len([name for name in os.listdir(jpgs_folder) if os.path.isfile(os.path.join(jpgs_folder, name))])
buff=10
background = []
bbox = []
fname = []
diff_open = []
angle = []
count = 0
# For each jpeg file in the rotated folder, do the following:
for jpg in itertools.islice(sorted(jpgs_folder.rglob('*.jpg')), limit):
    print(f'Processing {jpg.name}')
    img = Image.open(jpg)
    best_angle = rotation_angle(img)
    diff0, background0, bbox0 = (trim(img, angle=best_angle, buff=buff, find_top = find_top[count]))
    bbox.append(bbox0)
    background.append(background0)
    fname.append(jpg.name)
    diff_open.append(diff0)
    angle.append(best_angle)
    count = count + 1

print('Images successfully processed')

Processing Démonomanie I.1_image_0000.jpg
Processing Démonomanie I.1_image_0001.jpg
Processing Démonomanie I.1_image_0002.jpg
Processing Démonomanie I.1_image_0003.jpg
Processing Démonomanie I.1_image_0004.jpg
Processing Démonomanie I.1_image_0005.jpg
Processing Démonomanie I.1_image_0006.jpg
Processing Démonomanie I.1_image_0007.jpg
Processing Démonomanie I.1_image_0008.jpg
Processing Démonomanie I.1_image_0009.jpg
Processing Démonomanie I.1_image_0010.jpg
Processing Démonomanie I.1_image_0011.jpg
Images successfully processed


In [11]:
# Update meta with diffs
diff_list = []
count = 0

for file in fname:
        diff_dict = dict()
        diff_dict["filename"] = fname[count]
        diff_dict["bbox"] = bbox[count]
        diff_dict["background"] = background[count]
        diff_dict["angle"] = angle[count]
        diff_list.append(diff_dict)
        count = count+1
            
for original_dict in meta:
    for update_dict in diff_list:
        if original_dict["filename"] == update_dict["filename"]:
            original_dict.update(update_dict)     
meta

[{'filename': 'Démonomanie I.1_image_0000.jpg',
  'side': 'right',
  'start_section': False,
  'find_top': False,
  'background': (221, 207, 177),
  'bbox': (0, 0, 3556, 5456),
  'angle': 0.0},
 {'filename': 'Démonomanie I.1_image_0001.jpg',
  'side': 'left',
  'start_section': False,
  'find_top': False,
  'background': (220, 203, 173),
  'bbox': (9, 0, 3556, 5226),
  'angle': 0.0},
 {'filename': 'Démonomanie I.1_image_0002.jpg',
  'side': 'right',
  'start_section': False,
  'find_top': False,
  'background': (222, 207, 178),
  'bbox': (0, 0, 3556, 5355),
  'angle': 0.0},
 {'filename': 'Démonomanie I.1_image_0003.jpg',
  'side': 'left',
  'start_section': False,
  'find_top': False,
  'background': (220, 204, 174),
  'bbox': (0, 0, 3556, 5275),
  'angle': 0.0},
 {'filename': 'Démonomanie I.1_image_0004.jpg',
  'side': 'right',
  'start_section': False,
  'find_top': False,
  'background': (222, 208, 178),
  'bbox': (0, 0, 3556, 5306),
  'angle': -0.5},
 {'filename': 'Démonomanie I.1_

In [15]:
#Bands creation workflow
bheight = 50
band_dict = []
limit = len([name for name in os.listdir(jpgs_folder) if os.path.isfile(os.path.join(jpgs_folder, name))])

count = 0
# For each jpeg file in the rotated folder, do the following:
for jpg in itertools.islice(jpgs_folder.rglob('*.jpg'), limit):
    print(f'Processing {jpg.name}')
    band0 =  get_bands(diff_open[count], bheight=bheight)
    band_dict.append(band0)
    
    count = count + 1

print('Bands successfully created')

Processing Démonomanie I.1_image_0002.jpg
Processing Démonomanie I.1_image_0005.jpg
Processing Démonomanie I.1_image_0004.jpg
Processing Démonomanie I.1_image_0001.jpg
Processing Démonomanie I.1_image_0011.jpg
Processing Démonomanie I.1_image_0009.jpg
Processing Démonomanie I.1_image_0008.jpg
Processing Démonomanie I.1_image_0006.jpg
Processing Démonomanie I.1_image_0010.jpg
Processing Démonomanie I.1_image_0003.jpg
Processing Démonomanie I.1_image_0007.jpg
Processing Démonomanie I.1_image_0000.jpg
Bands successfully created


In [16]:
#Bands application workflow
limit = len([name for name in os.listdir(jpgs_folder) if os.path.isfile(os.path.join(jpgs_folder, name))])
count = 0
cut_list = []
pad = 10
allow = (0.1, 0.3)
freq = 0.8
minfreq = 0.1

for jpg in itertools.islice(sorted(jpgs_folder.rglob('*.jpg')), limit):
    print(f'Processing {jpg.name}')
    cut0 = simp_bd(band_dict=band_dict[count], diff=diff_open[count], side=side[count], width=diff_open[count].size[0],
                   pad=pad, allow = allow, freq = freq, minfreq = minfreq)
    cut_list.append(cut0)
    count = count + 1

print('Bands successfully applied')
cut_list[0:3]

Processing Démonomanie I.1_image_0000.jpg
Processing Démonomanie I.1_image_0001.jpg
Processing Démonomanie I.1_image_0002.jpg
Processing Démonomanie I.1_image_0003.jpg
Processing Démonomanie I.1_image_0004.jpg
Processing Démonomanie I.1_image_0005.jpg
Processing Démonomanie I.1_image_0006.jpg
Processing Démonomanie I.1_image_0007.jpg
Processing Démonomanie I.1_image_0008.jpg
Processing Démonomanie I.1_image_0009.jpg
Processing Démonomanie I.1_image_0010.jpg
Processing Démonomanie I.1_image_0011.jpg
Bands successfully applied


[3556, 0, 3556]

In [18]:
# Update meta with cuts
count = 0
cut_update=[]
for file in fname:
        cut_dict = dict()
        cut_dict["filename"] = fname[count]
        cut_dict["cut"] = cut_list[count]
        cut_update.append(cut_dict)
        count = count+1
            
for original_dict in meta:
    for update_dict in cut_update:
        if original_dict["filename"] == update_dict["filename"]:
            original_dict.update(update_dict)     
meta[0:3]

[{'filename': 'Démonomanie I.1_image_0000.jpg',
  'side': 'right',
  'start_section': False,
  'find_top': False,
  'background': (221, 207, 177),
  'bbox': (0, 0, 3556, 5456),
  'angle': 0.0,
  'cut': 3556},
 {'filename': 'Démonomanie I.1_image_0001.jpg',
  'side': 'left',
  'start_section': False,
  'find_top': False,
  'background': (220, 203, 173),
  'bbox': (9, 0, 3556, 5226),
  'angle': 0.0,
  'cut': 0},
 {'filename': 'Démonomanie I.1_image_0002.jpg',
  'side': 'right',
  'start_section': False,
  'find_top': False,
  'background': (222, 207, 178),
  'bbox': (0, 0, 3556, 5355),
  'angle': 0.0,
  'cut': 3556}]

In [19]:
# create a bbox for the entire diff image
out_bbox = []
count = 0
for jpg in itertools.islice(sorted(jpgs_folder.rglob('*.jpg')), limit):
    out_bbox0 = [0, 0] + list(diff_open[count].size)
    # Set the side value so that the function 
    # knows which bbox dimension to "cut"
    side_dict = {"left":0, "right":2}
    # Set the outer edge of the page equal to cut, 
    # rather than the original outer edge from diff
    # This creates a bbox that is cropped to include 
    # only the main text areas of diff
    side_0 = side[count]
    out_bbox0[side_dict[side_0]] = cut_list[count]
    out_bbox.append(out_bbox0)
    count = count + 1

print('Bbox margins successfully defined')
out_bbox[0:3]

Bbox margins successfully defined


[[0, 0, 3556, 5456], [0, 0, 3547, 5226], [0, 0, 3556, 5355]]

In [20]:
#Creating cropped image boundaries
total_bbox = []
count = 0
for jpg in itertools.islice(sorted(jpgs_folder.rglob('*.jpg')), limit):
    total_bbox0 = combine_bbox(bbox[count], out_bbox[count])
    total_bbox0=list(total_bbox0)
    total_bbox.append(total_bbox0)
    count = count + 1

print('Bbox dimensions successfully adjusted')
total_bbox[0:3]

Bbox dimensions successfully adjusted


[[0, 0, 3556, 5456], [9, 0, 3556, 5226], [0, 0, 3556, 5355]]

In [21]:
# Extract angle
angle_list = [d['angle'] for d in meta]
print(angle_list[0:3])

[0.0, 0.0, 0.0]


In [22]:
#metadata collected
filename_list = [d['filename'] for d in meta]
side_list = [d['side'] for d in meta]
meta_final = []
count = 0
for jpg in itertools.islice(sorted(jpgs_folder.rglob('*.jpg')), limit):
        meta_list = [filename_list[count], angle_list[count], side_list[count], cut_list[count]]
        meta_list.extend(background[count])
        meta_list.extend(total_bbox[count])
        meta_final.append(meta_list)
        count = count+1
 
meta_final[0:3]

[['Démonomanie I.1_image_0000.jpg',
  0.0,
  'right',
  3556,
  221,
  207,
  177,
  0,
  0,
  3556,
  5456],
 ['Démonomanie I.1_image_0001.jpg',
  0.0,
  'left',
  0,
  220,
  203,
  173,
  9,
  0,
  3556,
  5226],
 ['Démonomanie I.1_image_0002.jpg',
  0.0,
  'right',
  3556,
  222,
  207,
  178,
  0,
  0,
  3556,
  5355]]

In [23]:
#metadata csv
headers = ["filename","angle","side","cut","backR","backG","backB",
           "bbox1","bbox2","bbox3","bbox4"]
with open(f'{outDir}/{doc_name}_metadata.csv',"w",newline="") as outfile:
    writer=csv.writer(outfile)
    writer.writerow(headers)
    for row in meta_final:
        writer.writerow(row)

In [61]:
df = pd.read_csv(f'{outDir}/{doc_name}_metadata.csv')
df.head()

,filename,angle,side,cut,backR,backG,backB,bbox1,bbox2,bbox3,bbox4
0,Démonomanie I.1_image_0000.jpg,0.0,right,3556,221,207,177,0,0,3556,5456
1,Démonomanie I.1_image_0001.jpg,0.0,left,0,220,203,173,9,0,3556,5226
2,Démonomanie I.1_image_0002.jpg,0.0,right,3556,222,207,178,0,0,3556,5355
3,Démonomanie I.1_image_0003.jpg,0.0,left,0,220,204,174,0,0,3556,5275
4,Démonomanie I.1_image_0004.jpg,-0.5,right,3556,222,208,178,0,0,3556,5306


In [35]:
# Initialize variables
metalist = f"{outDir}/{doc_name}_metadata.csv"
csv = pd.read_csv(metalist)
n = 10

In [37]:
pool = []
csvf = csv[csv["filename"].str.startswith(doc_name)].set_index("filename")
csvf['file'] = csvf.index
for row in csvf.itertuples():
    pool.append(os.path.normpath(os.path.join(dirpath, row.file)))

pool = sample(pool, n)

pool

['images/Démonomanie I.1_image_0003.jpg',
 'images/Démonomanie I.1_image_0001.jpg',
 'images/Démonomanie I.1_image_0002.jpg',
 'images/Démonomanie I.1_image_0007.jpg',
 'images/Démonomanie I.1_image_0010.jpg',
 'images/Démonomanie I.1_image_0000.jpg',
 'images/Démonomanie I.1_image_0006.jpg',
 'images/Démonomanie I.1_image_0011.jpg',
 'images/Démonomanie I.1_image_0004.jpg',
 'images/Démonomanie I.1_image_0005.jpg']

In [40]:
#Get images for files in sample, cut margins and make a test list
imgs = []
results = []

for img in pool:

    #get image name
    name = os.path.split(img)[1]
    print('Testing', name)
    #get values for cutting margins
    rotate = csvf.loc[name]["angle"]
    left = csvf.loc[name]["bbox1"]
    up = csvf.loc[name]["bbox2"]
    right = csvf.loc[name]["bbox3"]
    lower = csvf.loc[name]["bbox4"]
    bkgcol = (csvf.loc[name]["backR"], csvf.loc[name]["backG"], csvf.loc[name]["backB"])

    #cut the margins
    img = cutMarg(img = img, rotate = rotate, left = left, up = up, right = right,
                 lower = lower, border = 0, bkgcol = bkgcol)


    #add the new image to the list
    imgs.append(img)

    #perform an OCR test on the new image and add the results to the list
    results.append(OCRtestImg(img))

Testing Démonomanie I.1_image_0003.jpg
Testing Démonomanie I.1_image_0001.jpg
Testing Démonomanie I.1_image_0002.jpg
Testing Démonomanie I.1_image_0007.jpg
Testing Démonomanie I.1_image_0010.jpg
Testing Démonomanie I.1_image_0000.jpg
Testing Démonomanie I.1_image_0006.jpg
Testing Démonomanie I.1_image_0011.jpg
Testing Démonomanie I.1_image_0004.jpg
Testing Démonomanie I.1_image_0005.jpg


In [43]:
#create a testList object with the  images and results
testSample = testList(imgs, results)

In [44]:
#list of available tests
print('conTests = ["color", "brightness", "contrast", "autocontrast", "sharpness"]')
print('boolTests = ["invert", "blur", "sharpen", "smooth", "xsmooth"]')

conTests = ["color", "brightness", "contrast", "autocontrast", "sharpness"]
boolTests = ["invert", "blur", "sharpen", "smooth", "xsmooth"]


In [46]:
#set up a dict of recommended adjustments and perform tests
adjustments = { "volume": doc_name, "color": 1.0, "invert": False, 
                "autocontrast": 0, "blur": False, "sharpen": False, 
                "smooth": False, "xsmooth": False }

In [47]:
#color test
print("color test")
testRes = testSample.adjustTest("color", levels = [1,.75,.5,.25,0])

color test



                                color1  color0.75  color0.5  color0.25  \
Démonomanie I.1_image_0003.jpg     122        138       145        139   
Démonomanie I.1_image_0001.jpg     136        128       132        139   
Démonomanie I.1_image_0002.jpg     139        128       127        130   
Démonomanie I.1_image_0007.jpg     126        128       132        137   
Démonomanie I.1_image_0010.jpg     143        133       134        143   
Démonomanie I.1_image_0000.jpg     106        108       105        104   
Démonomanie I.1_image_0006.jpg     128        127       130        137   
Démonomanie I.1_image_0011.jpg     127        127       119        126   
Démonomanie I.1_image_0004.jpg     128        128       131        131   
Démonomanie I.1_image_0005.jpg     147        147       137        141   

                                color0  totaltok  
Démonomanie I.1_image_0003.jpg     157       214  
Démonomanie I.1_image_0001.jpg     137       235  
Démonomanie I.1_im

In [48]:
best = float(testRes["best_adjustment"].replace("color", ""))
if best != 1.0:
    testSample = testSample.adjustImg(color = best)
    adjustments["color"] = best
    

In [49]:
#autocontrast test
print("autocontrast test")
testRes = testSample.adjustTest("autocontrast", levels = [0,2,4,6,8])
best = float(testRes["best_adjustment"].replace("autocontrast", ""))
if best != 0.0:
    testSample = testSample.adjustImg(autocontrast = best)
    adjustments["autocontrast"] = best

autocontrast test



                                autocontrast0  autocontrast2  autocontrast4  \
Démonomanie I.1_image_0003.jpg            138            133            142   
Démonomanie I.1_image_0001.jpg            128            131            134   
Démonomanie I.1_image_0002.jpg            128            130            130   
Démonomanie I.1_image_0007.jpg            128            126            126   
Démonomanie I.1_image_0010.jpg            133            134            140   
Démonomanie I.1_image_0000.jpg            108            115            107   
Démonomanie I.1_image_0006.jpg            127            128            131   
Démonomanie I.1_image_0011.jpg            127            126            133   
Démonomanie I.1_image_0004.jpg            128            127            132   
Démonomanie I.1_image_0005.jpg            147            139            140   

                                autocontrast6  autocontrast8  totaltok  
Démonomanie I.1_image_0003.jpg      

In [50]:
#blur test
print("blur test")
testRes = testSample.adjustTest("blur")
if testRes["best_adjustment"] == "blurTrue":
    testSample = testSample.adjustImg(blur = True)
    adjustments["blur"] = True

blur test



                                blurTrue  blurFalse  totaltok
Démonomanie I.1_image_0003.jpg       132        127       220
Démonomanie I.1_image_0001.jpg       135        127       238
Démonomanie I.1_image_0002.jpg       119        128       260
Démonomanie I.1_image_0007.jpg       131        121       215
Démonomanie I.1_image_0010.jpg       134        134       242
Démonomanie I.1_image_0000.jpg        98        102       174
Démonomanie I.1_image_0006.jpg       141        129       212
Démonomanie I.1_image_0011.jpg       124        127       220
Démonomanie I.1_image_0004.jpg       128        132       221
Démonomanie I.1_image_0005.jpg       149        149       255

TOTAL READABILITY
blurTrue: 42.8
blurFalse: 43.465


In [51]:
#sharpen test
print("sharpen test")
testRes = testSample.adjustTest("sharpen")
if testRes["best_adjustment"] == "sharpenTrue":
    testSample = testSample.adjustImg(sharpen = True)
    adjustments["sharpen"] = True

sharpen test



                                sharpenTrue  sharpenFalse  totaltok
Démonomanie I.1_image_0003.jpg          130           127       220
Démonomanie I.1_image_0001.jpg          134           127       238
Démonomanie I.1_image_0002.jpg          132           128       260
Démonomanie I.1_image_0007.jpg          131           121       215
Démonomanie I.1_image_0010.jpg          133           134       242
Démonomanie I.1_image_0000.jpg          105           102       174
Démonomanie I.1_image_0006.jpg          128           129       212
Démonomanie I.1_image_0011.jpg          125           127       220
Démonomanie I.1_image_0004.jpg          137           132       221
Démonomanie I.1_image_0005.jpg          140           149       255

TOTAL READABILITY
sharpenTrue: 42.623
sharpenFalse: 43.465


In [52]:
#smooth test
print("smooth test")
testRes = testSample.adjustTest("smooth")
if testRes["best_adjustment"] == "smoothTrue":
    testSample = testSample.adjustImg(smooth = True)
    adjustments["smooth"] = True

    #xsmooth test
    print("xsmooth test")
    testRes = testSample.adjustTest("xsmooth")
    if testRes["best_adjustment"] == "xsmoothTrue":
        testSample = testSample.adjustImg(xsmooth = True)
        adjustments["xsmooth"] = True

smooth test



                                smoothTrue  smoothFalse  totaltok
Démonomanie I.1_image_0003.jpg         132          127       220
Démonomanie I.1_image_0001.jpg         137          127       238
Démonomanie I.1_image_0002.jpg         128          128       260
Démonomanie I.1_image_0007.jpg         123          121       215
Démonomanie I.1_image_0010.jpg         131          134       242
Démonomanie I.1_image_0000.jpg         105          102       174
Démonomanie I.1_image_0006.jpg         136          129       212
Démonomanie I.1_image_0011.jpg         125          127       220
Démonomanie I.1_image_0004.jpg         129          132       221
Démonomanie I.1_image_0005.jpg         141          149       255

TOTAL READABILITY
smoothTrue: 42.977
smoothFalse: 43.465


In [53]:
#post-test adjustment levels
adjustments
adjustments_list = [*adjustments.values()]
adjustments_list

['Démonomanie I.1', 0.75, False, 8.0, False, False, False, False]

In [60]:
#adjustments csv
headers = ["volume","color","invert","autocontrast","blur","sharpen","smooth",
           "xsmooth"]
with open(f"{outDir}/{doc_name}_adjustments.csv","w",newline="") as outfile:
    writer=csv.writer(outfile)
    writer.writerow(headers)
    writer.writerow(adjustments_list)

In [64]:
#Set paths for metadata files
margdata = f"{outDir}/{doc_name}_metadata.csv"
adjdata = f"{outDir}/{doc_name}_adjustments.csv"

#Read metadata files into memory
#Read csvs
margcsv = pd.read_csv(margdata)
adjcsv = pd.read_csv(adjdata)

In [65]:
#merge csvs
margcsv['key'] = 0
adjcsv['key'] = 0

# Perform the Cartesian product
fulldata = pd.merge(margcsv, adjcsv, on='key')

# Drop the temporary key column
fulldata.drop('key', axis=1, inplace=True)

# Save the combined dataframe to a new CSV file
fulldata.to_csv(f'{outDir}/{doc_name}_fulldata.csv', index=False)

In [6]:
#create OCR files
fulldata_path = f"{outDir}/{doc_name}_fulldata.csv"
fulldata = pd.read_csv(fulldata_path)
#Loop through section
for row in fulldata.itertuples():

    img = os.path.normpath(os.path.join(dirpath, row.filename))
    print(datetime.now().strftime("%H:%M") + " Processing " + row.filename + "...")

    #set up margin cutting
    cuts = {"rotate" : row.angle,
            "left" : row.bbox1,
            "up" : row.bbox2,
            "right" : row.bbox3,
            "lower" : row.bbox4,
            "border" : 0,
            "bkgcol" : (row.backR, row.backG, row.backB)}

    #set up image adjustment            
    adjustments = {"color": row.color, 
                   "autocontrast": row.autocontrast,
                   "blur": row.blur,
                   "sharpen": row.sharpen,
                   "smooth": row.smooth,
                   "xsmooth": row.xsmooth}

    #OCR the image
    tsvOCR((adjustImg(cutMarg(img, **cuts), **adjustments)), 
           savpath = os.path.normpath(os.path.join(outDir, doc_name + ".txt")),
           tsvfile = doc_name + "_data.tsv")

print('Done OCR job')

18:01 Processing Démonomanie I.1_image_0000.jpg...
18:01 Processing Démonomanie I.1_image_0001.jpg...
18:01 Processing Démonomanie I.1_image_0002.jpg...
18:02 Processing Démonomanie I.1_image_0003.jpg...
18:02 Processing Démonomanie I.1_image_0004.jpg...
18:02 Processing Démonomanie I.1_image_0005.jpg...
18:02 Processing Démonomanie I.1_image_0006.jpg...
18:02 Processing Démonomanie I.1_image_0007.jpg...
18:02 Processing Démonomanie I.1_image_0008.jpg...
18:03 Processing Démonomanie I.1_image_0009.jpg...
18:03 Processing Démonomanie I.1_image_0010.jpg...
18:03 Processing Démonomanie I.1_image_0011.jpg...
Done OCR job
